Conditional correlation analysis

In [22]:
# python modules
import copy
import random
import re
import glob
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
from scipy import stats

In [23]:
# custom modules
import graphics_edit as graphics
import load_file_edit as load_file
import Interval_dict
import statis_edit as statis

In [24]:
# matplotlib setting
%matplotlib inline
mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["axes.facecolor"] = "white"
mpl.rcParams["savefig.facecolor"] = "white"

In [25]:
### parameters
cell_org = {'H1':'human',
            'GM':'human',
            'mCD8T:WT':'mouse',
            'mCD8T:DFMO':'mouse',
            'mCD8T:ODCKO':'mouse'}

cell_chrnames = {'H1':['chr%s' % (i) for i in range(1, 23)] + ['chrX', 'chrY'],
                 'GM':['chr%s' % (i) for i in range(1, 23)] + ['chrX'],
                 'mCD8T:WT':['chr%s' % (i) for i in range(1, 20)] + ['chrX'],
                 'mCD8T:DFMO':['chr%s' % (i) for i in range(1, 20)] + ['chrX'],
                 'mCD8T:ODCKO':['chr%s' % (i) for i in range(1, 20)] + ['chrX']}

In [27]:
### set data information (fname/field) and key
path = '/Users/sangwoopark/jhu_rockfish/2024_01_05_GEO/processed_files/'

dinfo_dkey = {'H1_NCP_sp_1rep_deep_chr1_score_table.gtab.gz':None}

In [28]:
### load gtab file
dkey_ID_value = {}
for fkey in dinfo_dkey:
    field_dkey = dinfo_dkey[fkey]

    if field_dkey == None:
        field_choices = None
    else:
        field_choices = field_dkey.keys()
    
    for fname in glob.glob(path + '*'):
        if not re.match(fkey, fname.rsplit('/')[-1]):
            continue
        print "loading %s" % (fname.rsplit('/')[-1])

        field_ID_value = load_file.read_gtab(fname,
                                             mode='col',
                                             field_choices=field_choices)

        if field_dkey == None:
            field_dkey = {field:field for field in field_ID_value.keys()}

        for field, dkey in field_dkey.items():
            ID_value = field_ID_value[field]
            if dkey not in dkey_ID_value:
                dkey_ID_value[dkey] = {}
            dkey_ID_value[dkey].update(ID_value)
            

loading H1_NCP_sp_1rep_deep_chr1_score_table.gtab.gz


In [29]:
# change dkey name
dkey_ID_value['AT content'] = copy.deepcopy(dkey_ID_value['ATcontent'])
del dkey_ID_value['ATcontent']

In [30]:
### compute sequence features
# methylation density
dkey_ID_value['meCpG density'] = statis.get_fract_dict(dkey_ID_value['CNumber(CpG)'],
                                                       dkey_ID_value['meCNumber(CpG)'])
dkey_ID_value['meCHG density'] = statis.get_fract_dict(dkey_ID_value['CNumber(CHG)'],
                                                       dkey_ID_value['meCNumber(CHG)'])
dkey_ID_value['meCHH density'] = statis.get_fract_dict(dkey_ID_value['CNumber(CHH)'],
                                                       dkey_ID_value['meCNumber(CHH)'])


In [31]:
### compute sequence features
# mean poly GC length
ID_polyGC = {}
for ID, seq in dkey_ID_value['Sequence'].items():
    num_pos = statis.polynt_count(seq.upper(), nts='GC', pos=True)
    mean_len, count = 0.0, 0.0
    for num, pos in num_pos.items():
        mean_len += len(pos)*num
        count += len(pos)
    ID_polyGC[ID] = mean_len/count

dkey_ID_value['poly-G/C length'] = ID_polyGC

del dkey_ID_value['Sequence']
del ID_polyGC


In [32]:
### select feature set and target for analysis
seq_features = ['AT content', 'poly-G/C length']
seq_features = []

epigenetic_features = ['meCpG density', 'meCHG density', 'meCHH density', 'H2AFZ', 'H2AK5ac', 'H2BK120ac', 'H2BK12ac', 'H2BK15ac', 'H2BK20ac', 'H2BK5ac', 'H3K14ac', 'H3K18ac', 'H3K23ac', 'H3K23me2', 'H3K27ac', 'H3K27me3', 'H3K36me3', 'H3K4ac', 'H3K4me1', 'H3K4me2', 'H3K4me3', 'H3K56ac', 'H3K79me1', 'H3K79me2', 'H3K9ac', 'H3K9me3', 'H4K20me1', 'H4K5ac', 'H4K8ac', 'H4K91ac']

features = seq_features + epigenetic_features

In [33]:
### set target for analysis
target = 'H1_NCP_sp_8_1rep_deep'

In [ ]:
### binning the features and get state
ID_score = dkey_ID_value[target]
IDs = ID_score.keys()

X = [[] for i in range(len(IDs))]
for feature in features:
    values = [dkey_ID_value[feature][ID] for ID in IDs]
    min_value = min(values)
    max_value = max(values)
    for i in range(len(IDs)):
        value = values[i]
        re_value = float(value-min_value)/max_value
        X[i].append(re_value)
X = sparse.csr_matrix(X)
del dkey_ID_value

In [ ]:
# None-negative matrix factorization
class_num = 10

try:
    with open("W.pickle", "rb") as f:
        W = pickle.load(f)
    with open("H.pickle", "rb") as f:
        H = pickle.load(f)

except:
    print "NMF start"
    model = NMF(n_components=class_num, init='random', random_state=0, verbose=True)
    W = model.fit_transform(X)
    H = model.components_
    print "NMF is done"

    with open("W.pickle", "wb") as f:
        pickle.dump(W, f)
    with open("H.pickle", "wb") as f:
        pickle.dump(H, f)

In [ ]:
# post-analysis of NMF
cID_prog = []
for i in range(class_num):
    cID_prog.append(H[i])

ID_cID = {}
cID_IDs = [[] for i in range(class_num)]
for i in range(len(IDs)):
    ID = IDs[i]
    cID = np.argmax(W[i])
    ID_cID[ID] = cID
    cID_IDs[cID].append(ID)

cID_scores = [[] for i in range(class_num)]
for i in range(len(cID_IDs)):
    for ID in cID_IDs[i]:
        score = ID_score[ID]
        cID_scores[i].append(score)x

In [ ]:
# sort according to condensability
score_cID = sorted([(np.median(cID_scores[cID]), cID) for cID in range(len(cID_scores))])
cID_list = [cID for score, cID in score_cID]
cID_newcID = {cID_list[i]:i for i in range(len(cID_list))}
ID_newcID = {}
for ID in IDs:
    cID = ID_cID[ID]
    newcID = cID_newcID[cID]
    ID_newcID[ID] = newcID
    
with open("NMF_sorted_cID.pickle", "wb") as f:
    pickle.dump(ID_newcID, f)

In [ ]:
# plot property matrix
def plot_NMF_basis_matrix (basis_matrix,
                           basis_idxs,
                           features,
                           feature_cmaps=None,
                           xlabel='Property class',
                           title=None,
                           fig_width=5,
                           fig_height=6,
                           ax=None,
                           save=False,
                           note=''):

    if not ax:
        fig, ax = plt.subplots(nrows=1,
                               ncols=1,
                               figsize=(fig_width, fig_height))
        make_fig = True
    else:
        make_fig = False

    assert len(basis_idxs) == len(basis_matrix)
    assert len(features) == len(basis_matrix[0])

    if not feature_cmaps:
        feature_cmaps = ['Greys'] * len(features)
        
    elif type(feature_cmaps) == list:
        if len(feature_cmaps) < len(features):
            repeat = float(len(features)) / len(feature_cmaps)
            repeat = int(math.ceil(repeat))
            feature_cmaps = feature_cmaps * repeat
            feature_cmaps = feature_cmaps[:len(features)]

    elif type(feature_cmaps) == str:
        feature_cmaps = [feature_cmaps] * len(features)
        
    imgs = []
    for i in range(len(features)):
        img = np.zeros((len(features), len(basis_idxs)))
        img[:] = np.nan
        for j in range(len(basis_idxs)):
            idx = basis_idxs[j]
            img[i][j] = basis_matrix[idx][i]
        imgs.append(img)

    for img, cmap in zip(imgs, feature_cmaps):
        plt.imshow(img, cmap=cmap, aspect='auto')

    for i in range(len(basis_idxs)):
        idx = basis_idxs[i]
        for j in range(len(features)):
            mean = np.mean(basis_matrix[:,j])
            std = np.std(bais_matrix[:,j])
            value = basis_matrix[idx][j]
            if value > mean + std:
                color = 'white'
            else:
                color = 'black'
            ax.text(i,
                    j,
                    str(round(value, 2)),
                    ha="center",
                    va="center",
                    fontsize=8,
                    color=color)

    ax.set_xticks(range(len(basis_idxs)))
    ax.set_xticklabels(range(1, len(basis_idxs)+1),
                       fontsize=8)

    ax.set_yticks(range(len(features)))
    ax.set_yticklabels(features,
                       fontsize=8)

    if xlabel:
        ax.set_xlabel(xlabel, fontsize=10)

    if title:
        ax.set_title(title, fontsize=12)

    if make_fig:
        if save:
            plt.savefig("NMF_basis_matrix_%s.svg" % (note),
                        format='svg',
                        bbox_inches='tight')
        else:
            plt.tight_layout()
            plt.show()    
        plt.close()

    return ax

In [ ]:
# plot property matrix
cmaps = ['Reds', 'Blues', 'Greens', 'Purples', 'Oranges',  'Greys']
plot_NMF_basis_matrix (H,
                       cID_list,
                       features,
                       feature_cmaps=cmaps,
                       xlabel='Property class',
                       title=None,
                       fig_width=5,
                       fig_height=6,
                       ax=None,
                       save=False,
                       note='H1_NCP')

In [ ]:
# plot condensability of each property class
fig = plt.figure(figsize=(5,3))
plt.boxplot([cID_scores[cID] for cID in cID_list], 0, "")
plt.title("Condensability by property class", fontsize=12)
#plt.xlabel('Property class', fontsize=10)
plt.ylabel('Condensability (A.U.)', fontsize=10)
plt.gca().tick_params(axis='both', which='major', labelsize=8)
plt.gca().tick_params(axis='both', which='minor', labelsize=8)
plt.savefig("condensability_by_class.svg", format='svg', bbox_inches='tight')
#plt.xticks(range(1, len(cID_scores)+1), range(1, len(cID_scores)+1))
#plt.tight_layout()
#plt.savefig("anatomy_pbox.png")
#plt.show()
plt.close()

In [ ]:
# print NMF result
f = open("NMF_property_class.txt", 'w')
print >> f, "Class#" + "\t" + "\t".join(names)
for i in range(len(cID_prog)):
    print >> f, str(i+1) + "\t" + "\t".join([str(value) for value in cID_prog[i]])
f.close()

f = open("NMF_NCPClass.txt", 'w')
print >> f, "ID" + "\t" + "Class#"
for ID in ID_cID:
    print >> f, str(ID) + "\t" + str(ID_cID[ID]+1)
f.close()